In [11]:
pip install selenium beautifulsoup4 pandas fake_useragent colorama openpyxl


[notice] A new release of pip is available: 24.0 -> 24.1.2

[notice] To update, run: python.exe -m pip install --upgrade pip

In [2]:
chrome = False
chrome = True

if chrome:
    from selenium.webdriver.chrome.options import Options
else:
    from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
sleep = time.sleep
time = time.time
import re
from fake_useragent import UserAgent
import colorama

################################### Progress Bar ###################################
colorama.init()
RED = colorama.Fore.LIGHTRED_EX
CYAN = colorama.Fore.CYAN
GREEN = colorama.Fore.LIGHTGREEN_EX
YELLOW = colorama.Fore.YELLOW
LIGHTYELLOW = colorama.Fore.LIGHTYELLOW_EX
MAGENTA = colorama.Fore.LIGHTMAGENTA_EX
DARKGREEN = colorama.Fore.GREEN
BLUE = colorama.Fore.BLUE
RESET = colorama.Fore.RESET

def convertSeconds(seconds):
    seconds = float(seconds)
    hours = int(seconds / 3600)
    seconds %= 3600
    minutes = int(seconds / 60)
    seconds %= 60

    time_str = f"[{hours}:{minutes}:{seconds:.2f}]"
    return time_str

def progressBar(total, current, start_time):
    current += 1
    
    empty = RED+'░'
    filled = GREEN+'█'
    border = BLUE+'|'

    percent = (current/total)*100
    filled_count = int(percent)
    empty_count = 100-filled_count

    try:
        t = time()
        time_left = (t-start_time)*(total/current) - (t-start_time)
    except Exception as e:
        print(f'Error in calculating time left: {e}')
        time_left = 0

    time_left = convertSeconds(time_left)
    print(f'\r{border}{filled*filled_count}{empty*empty_count}{border}{YELLOW} {percent:.2f}% [{current}/{total}] {CYAN}{time_left} {RESET}',end='')

    if percent == 100:
        time_left = convertSeconds(time()-start_time)
        print(f'\r{border}{filled*filled_count}{empty*empty_count}{border}{YELLOW} {percent:.2f}% [{current}/{total}] {CYAN}{time_left} {MAGENTA}\n',end=RESET)

################################### Main Functionality ###################################

def setup():
    global action
    user = UserAgent().random
    options = Options()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--no-sandbox")
    # options.add_argument("--headless")  # Try running without headless mode to debug
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-crash-reporter")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-in-process-stack-traces")
    options.add_argument("--disable-logging")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--incognito")
    options.add_argument("--log-level=3")
    options.add_argument("--output=/dev/null")
    options.add_argument(f"user-agent={user}")

    # Add DNS server option
    options.add_argument("--dns-prefetch-disable")

    if chrome:
        driver = webdriver.Chrome(options=options)
    else:
        driver = webdriver.Firefox(options=options)
    driver.maximize_window()
    action = ActionChains(driver)

    return driver

from bs4 import BeautifulSoup
def get_data(element):
    html = element.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.get_text("\n", strip=True)
    return data + '\n'

def click(driver, element):
    driver.execute_script("arguments[0].click();", element)

def get_price(value):
    if value:
        value = value.replace(',','').replace(' ','')
        value = re.search(r'\d+\.?\d*',value)
        if value:
            value = value.group()
            value = eval(value)
        else:
            return ''
    return value

data = {
    'id': [],
    'url':[],
    'name':[],
    'price':[],
    'price unit':[],
    'code':[],
    'duration':[],
    'slot date':[],
    'slot time':[],
}

driver = setup()
mainUrl = 'https://www.exitcertified.com/search/SOLRSearchForm?terms=EC-COUNCIL&action_SOLRSearchForm_submit=Search&locale=en_US&RevertFilterAllowed=0&HasSearched=1&sort=Title_textSort%3A0&page=1&filter=+Course'
driver.get(mainUrl)
sleep(5)

urls = [a.get_attribute('href') for a in driver.find_elements(By.XPATH, '//h3[@class="solr-result-title md-typography md-typography--root md-typography-h6"]//a')]

total = len(urls)
Id = 0
current = 0
st = time()

for url in urls:
    driver.get(url)
    sleep(5)
    try:
        name = get_data(driver.find_element(By.XPATH, '//h1'))
    except:
        name = ''

    try:
        price = driver.find_element(By.XPATH, '//*[@class="price"]')
        price = get_data(price).replace(',','').replace(' ','')
        price = re.search(r'\d+\.?\d*',price).group()
        price_unit = 'USD'
    except:
        price = ''
        price_unit = ''

    try:
        code = get_data(driver.find_element(By.XPATH, '//*[@class="detail "]//*[@class="text"]'))
    except:
        code = ''
    
    try:
        duration = get_data(driver.find_element(By.XPATH, '//*[@class="detail duration"]//*[@class="text"]'))
    except:
        duration = ''

    slots = driver.find_elements(By.XPATH, '//*[@class="upcoming-event-schedule preview"]//li')

    if len(slots) == 0:
        slots = ['']

    for slot in slots:

        try:
            slot_date = get_data(slot.find_element(By.XPATH, './/*[@class="start"]'))
        except:
            slot_date = ''
        
        try:
            slot_time = get_data(slot.find_element(By.XPATH, './/*[@class="time-info"]'))
        except:
            slot_time = ''

        data['id'].append(Id)
        data['url'].append(url)
        data['name'].append(name)
        data['price'].append(price)
        data['price unit'].append(price_unit)
        data['code'].append(code)
        data['duration'].append(duration)
        data['slot date'].append(slot_date)
        data['slot time'].append(slot_time)

    Id += 1

    progressBar(total, current, st)
    current += 1

    # if current >= 2:
    #     break

driver.quit()

import pandas as pd
df = pd.DataFrame(data)
df.to_excel('exitCertified_ec-council.py.xlsx', index=False)
print('\nData saved to Excel')


|█████████████████████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 33.33% [1/3] [0:0:23.16] ██████████████████████████████████████████████████████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 66.67% [2/3] [0:0:13.56] ████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00% [3/3] [0:0:0.00] ████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00% [3/3] [0:0:37.54] 

Data saved to Excel


In [4]:
import pandas as pd

# Define the function to process the Excel file
def process_excel(input_file, output_file):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(input_file)

    # Print columns to understand their current state
    print("Original columns:", df.columns.tolist())

    # Split 'slot date' into 'start_date' and 'end_date'
    if 'slot date' in df.columns:
        df[['start_date', 'end_date']] = df['slot date'].str.split(' - ', expand=True)
    
    # Assign the correct values to the new columns
    df['slot_time'] = df['start_date']  # Move the original 'start_date' to 'slot_time'
    df['start_date'] = df['end_date']  # Move the original 'end_date' to 'start_date'
    df['end_date'] = df['slot_time']  # The original 'slot_time' should be in 'end_date'

    # Drop the original 'slot date' column
    df.drop(columns=['slot date'], inplace=True, errors='ignore')

    # Print columns to confirm changes
    print("Columns after processing:", df.columns.tolist())

    # Rename columns as specified, if the number of columns matches
    expected_columns = [
        'id',
        'url',
        'coursename',
        'price',
        'priceunit',
        'course_code',
        'duration',
        'start_date',
        'end_date',
        'slot_time'
    ]

    if len(df.columns) == len(expected_columns):
        df.columns = expected_columns
    else:
        print(f"Warning: Column length mismatch. DataFrame has {len(df.columns)} columns, but expected {len(expected_columns)}.")

    # Add static columns for 'brandname' and 'region'
    df['brandname'] = 'AWS'
    df['region'] = 'USA'

    # Reorder columns to match the desired output format
    final_columns = ['id', 'url', 'coursename', 'price', 'priceunit', 'course_code', 'duration', 'start_date', 'end_date', 'slot_time', 'brandname', 'region']
    df = df[final_columns]

    # Write the DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = 'D:/360DT/Course_Scanner/new/Code/exitCertified_ec-council.xlsx'
output_csv_file = 'output_file1.csv'
process_excel(input_excel_file, output_csv_file)


Original columns: ['id', 'url', 'name', 'price', 'price unit', 'code', 'duration', 'slot date', 'slot time']
Columns after processing: ['id', 'url', 'name', 'price', 'price unit', 'code', 'duration', 'slot time', 'start_date', 'end_date', 'slot_time']


KeyError: "['coursename', 'priceunit', 'course_code'] not in index"